# Lab 5
### ECE 4960 (Fall 2020)
<hr>

## Simulator
- The robot simulator simulates a differential drive, wheeled robot with a laser range finder, similar to our physical robot.
- The laser range finder is located at the front and center portion of the robot.
- In the simulator window, select "View->Data" or press \<d\> on your keyboard to visualize the range finder.

## Robot Class
It provides a control interface for the robot in the simulator. It setups up a communication channel to send/receive data to/from the smulator appication to perform the following operations:
- Get robot odometry pose
- Move the robot
- Get range finder data

In this lab, we will learn how to use the robot class to perform all the operations.

<hr>

## Import necessary modules

In [1]:
# Automatically reload changes in python modules
%load_ext autoreload
%autoreload 2

# Import robot class
from robot_interface import *

import time
import numpy as np
import rospy
from random import uniform

## Instantiate and Initialize an object of class Robot
**You need to run the below cell only once after the notebook is started. It intializes the communication channels to send/receive data to/from the simulator.**

In [2]:
robot = Robot()
robot.initialize()

Using python version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]


<hr>

## Get Robot Odometry Pose
To get the latest robot pose information, use the member function ***get_pose*** which returns a tuple of the format $(x, y, yaw)$ with units $(meters, meters, radians)$

In [3]:
# You may notice running this cell repeatedly even without 
# moving the robot produces different pose values.
# This is because the odometry is noisy.

pose = robot.get_pose()
print("Robot Odom Pose: ", pose)

Robot Odom Pose:  (0.016602344130348758, 0.028508222311406928, 0.6487408648678386)


## Move the robot
- The robot module also sends information to the simulator to set the robot velocity. <br>
- The robot is modelled as a differential drive system and takes in a linear and an angular velocity. <br>
- Use the member function ***set_vel(linear_velocity, angular_velocity)*** to set the command velocity for the robot. <br>
- The units for linear and angular velocities are meters/second and radians/second, respectively.

In [4]:
# Sets a linear velocity of 0.2m/s and angular velocity of 0.3radians/s and hence the robot executes a cirucular arc motion
# Try out different values
robot.set_vel(v=0.2,w=0.3)

### Publish a linear velocity of 0.5m/s for a 0.5 seconds interval and stop the robot.

In [5]:
robot.set_vel(0.5,0)
time.sleep(0.5)

robot.set_vel(0,0)

## Get Range Finder Data
To get data from the robot's range finder, use the **get_laser_data** member function. <br>
Press \<d\> in the simulator application or select "View->Data" to visualize the range finder in the simulator.

In [3]:
# You may notice running this cell repeatedly even when 
# the robot is standing still produces different range values.
# This is because the sensor is noisy.

robot.get_laser_data()

3.485929012298584

# Obstacle Avoidance
Use the member functions *get_laser_data()* and *set_vel()* to perform obstacle avoidance. <br> 
Write the obstacle avoidance code under the function definition *perform_obstacle_avoidance()*, which is then subsequently called in the last line of the cell. <br>

- Make sure your simulator is running :)
- Make changes to the function *perform_obstacle_avoidance(robot)*  and click the <button class='btn btn-default btn-xs'><i class="icon-step-forward fa fa-play"></i></button> to run the below cell.
- Since the function has a never ending loop, click the <button class='btn btn-default btn-xs'><i class='icon-stop fa fa-stop'></i></button> button to stop the cell i.e the obstacle avoidance code. <br>
- When the cell is running, you can notice a **[ \* ]** to the left of the cell.
- Make sure to stop the cell before making changes to the cell.
- **Dont forget to save your changes before quitting the jupyter server or closing this notebook.**


In [ ]:
## Simplified obstacle avoidance code using only right angles
def perform_obstacle_avoidance(robot):
    turnThisMuch = 91*3.14159/180;           # turn a little bit more than 90°
    while True:
        # Obstacle avoidance code goes here
        if robot.get_laser_data()<0.5:
            robot.set_vel(0.0,0.5)  # turn
            initialAngle = robot.get_pose()[2]
            angleTurned = 0
            # print("Turning...")             # for debugging
            while abs(angleTurned) < turnThisMuch: # turn, then read again
                angleTurned = robot.get_pose()[2]-initialAngle
                # print(angleTurned)          # for debugging
                time.sleep(0.001)
            robot.set_vel(0,0)
            # print("Finished turning")         # for debugging
        else:
            robot.set_vel(0.5,0)  # drive straight
        time.sleep(0.1)
           
perform_obstacle_avoidance(robot)

In [12]:
## Test the turning logic
robot.set_vel(0.0,0.5)  # turn
initialAngle = robot.get_pose()[2]
print(initialAngle)
angleTurned = 0
while angleTurned < 0.5: # turn about 30 degrees, then read again
    angleTurned = robot.get_pose()[2]-initialAngle
robot.set_vel(0,0)

1.00236018154719


In [ ]:
## A try at using arcs to avoid crashing into walls
def perform_obstacle_avoidance(robot):
    turnThisMuch = 45*3.14159/180;
    while True:
        # Obstacle avoidance code goes here
        if robot.get_laser_data()<0.5:
            robot.set_vel(0.0,0.5)  # turn
            initialAngle = robot.get_pose()[2]
            angleTurned = 0
            while angleTurned < turnThisMuch: # turn 45 degrees, then read again
                angleTurned = robot.get_pose()[2]-initialAngle
                #print(angleTurned)  # for debugging
                time.sleep(0.001)
            robot.set_vel(0,0)
        else:
            robot.set_vel(0.5,0.05)  # drive in a slight curve since we will
                                    # never be exactly parallel to a wall
        time.sleep(0.1)
           
perform_obstacle_avoidance(robot)